In [5]:
import os

In [6]:
%pwd

'C:\\deletable\\apps\\CoTProject\\code\\mlops'

In [7]:
os.chdir("../")

In [8]:
%pwd

'C:\\deletable\\apps\\CoTProject\\code'

In [9]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Hassen-Elmahrouk/CoTProject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Hassen-Elmahrouk"
os.environ["MLFLOW_TRACKING_PASSWORD"]="92f39719f5c80628067c008d65ad4bdfd797d941"

In [10]:
import tensorflow as tf

In [24]:
model = tf.keras.models.load_model("/deletable/apps/CoTProject/code/mlops/artifacts/training/model.h5")

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [26]:
from src.stawberryDeseaseClassifier.constants import *
from src.stawberryDeseaseClassifier.utils.common import read_yaml, create_directories, save_json

In [37]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="/deletable/apps/CoTProject/code/mlops/artifacts/training/model.h5",
            training_data="/deletable/apps/CoTProject/code/mlops/artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Hassen-Elmahrouk/CoTProject.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [38]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [39]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

   
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("/deletable/apps/CoTProject/code/mlops/artifacts/scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [40]:

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2023-11-11 20:07:25,573: INFO : common : yaml file: \deletable\apps\CoTProject\code\mlops\config\config.yaml loaded successfully]
[2023-11-11 20:07:25,590: INFO : common : yaml file: \deletable\apps\CoTProject\code\mlops\params.yaml loaded successfully]
[2023-11-11 20:07:25,597: INFO : common : created directory at: /deletable/apps/CoTProject/code/mlops/artifacts]
Found 139 images belonging to 2 classes.
139/139 [==============================] - 216s 2s/step - loss: 7.5675 - accuracy: 0.7050
[2023-11-11 20:11:04,197: INFO : common : json file saved at: \deletable\apps\CoTProject\code\mlops\artifacts\scores.json]


2023/11/11 20:11:14 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-11-11 20:11:17,569: WARNING : save : Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Hasse\AppData\Local\Temp\tmpcunkm2km\model\data\model\assets
[2023-11-11 20:11:18,820: INFO : builder_impl : Assets written to: C:\Users\Hasse\AppData\Local\Temp\tmpcunkm2km\model\data\model\assets]


MlflowException: API request to endpoint /api/2.0/mlflow/runs/get failed with error code 400 != 200. Response body: '"repo not associated with run"'